In [ ]:
import pandas as pd
from repsim.benchmark.paths import EXPERIMENT_RESULTS_PATH

## Group Separation

In [ ]:
# path = "/root/similaritybench/experiments/results/nlp_aug_mnli_full.csv"
# path = "/root/similaritybench/experiments/results/nlp_sc_mnli_full.csv"


cleaned_dfs = []
for path, setting, dataset in [
        ("/root/similaritybench/experiments/results/nlp_aug_sst2.csv", "aug", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_mem_sst2.csv", "mem", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_shortcut_sst2.csv", "sc", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_shortcut_mnli.csv", "sc", "mnli"),
        ("/root/similaritybench/experiments/results/nlp_aug_mnli.csv", "aug", "mnli"),
        ("/root/similaritybench/experiments/results/nlp_mem_mnli.csv", "mem", "mnli"),
    ]:
    print(setting, dataset)
    df = pd.read_csv(path)
    data = df.loc[2:].copy().reset_index(drop=True)
    print(data.columns)
    proper_name = {"quality_measure": "Similarity Measure", "AUPRC": "AUPRC", "violation_rate": "Violation Rate"}
    data.columns = [proper_name[col] for col in data.columns]
    data["Architecture"] = "BERT-Base"
    data.loc[:, "Violation Rate"] = data.loc[:, "Violation Rate"].astype(float)
    data.loc[:, "AUPRC"] = data.loc[:, "AUPRC"].astype(float)
    data = data.melt(
        id_vars=["Similarity Measure", "Architecture"],
        value_vars=["Violation Rate", "AUPRC"],
        var_name="Quality Metric",
        value_name="Score",
    )
    data["Setting"] = setting
    data["Dataset"] = dataset

    cleaned_dfs.append(data)

data = pd.concat(cleaned_dfs).reset_index(drop=True)
data.head()

In [ ]:
for setting in data.Setting.unique():
    pivot = pd.pivot(data[data.Setting == setting], index="Similarity Measure", columns=["Quality Metric", "Dataset"], values="Score")
    print(pivot.to_latex())

In [ ]:
pivot = data.pivot_table(index="Similarity Measure", columns=["Quality Metric", "Setting", "Dataset"], values="Score")
pivot


In [ ]:
styled = pd.io.formats.style.Styler(
    pivot,
    precision=3,
    caption="Ability to separate groups of BERT representations in different settings.",
)
styled

In [ ]:
latex_str = styled.to_latex(hrules=True, position="t", label="tab:nlp_groupsep")
latex_str = latex_str.split("\n")
latex_str = [r"\rowcolor{Gray}" + line  if i>=12 and (i-12)%2==0 else line for i, line in enumerate(latex_str[:-4])] + latex_str[-4:]
latex_str = "\n".join(latex_str)
print(latex_str)

In [ ]:
groupsep_data = data.copy()

## Output Correlation

In [ ]:
cleaned_dfs = []
for path, setting, dataset in [
    ("/root/similaritybench/experiments/results/correlation_nlp_aug_mnli_full.csv","aug","mnli"),
    ("/root/similaritybench/experiments/results/correlation_nlp_aug_sst2_full.csv","aug","sst2"),
    ("/root/similaritybench/experiments/results/correlation_nlp_mem_mnli_full.csv","mem","mnli"),
    ("/root/similaritybench/experiments/results/correlation_nlp_mem_sst2_full.csv","mem","sst2"),
    ("/root/similaritybench/experiments/results/correlation_nlp_sc_mnli_full.csv","sc","mnli"),
    ("/root/similaritybench/experiments/results/correlation_nlp_sc_sst2_full.csv","sc","sst2"),
    ]:
    df = pd.read_csv(path, index_col=0)
    df["Setting"] = setting
    df["Dataset"] = dataset
    cleaned_dfs.append(df)
data = pd.concat(cleaned_dfs).reset_index(drop=True)
data = data.rename(columns={"functional_similarity_measure": "Functional Similarity Measure", "similarity_measure": "Representational Similarity Measure", "quality_measure": "Quality Measure"})

In [ ]:
pivot = data[data["Quality Measure"]=="spearmanr"].pivot_table(index="Representational Similarity Measure",columns=["Functional Similarity Measure", "Quality Measure", "Setting", "Dataset"],values="corr")
# pivot = pivot.apply(abs)
pivot = pivot.sort_values(by="Representational Similarity Measure")
pivot

In [ ]:
styled = pd.io.formats.style.Styler(
    pivot,
    precision=3,
    caption="Absolute correlation between representational and functional similarity for BERT models.",
)
latex_str = styled.to_latex(hrules=True, position="t", label="tab:nlp_outputcorr", )
latex_str = latex_str.split("\n")
latex_str = [r"\rowcolor{Gray}" + line  if i>=12 and (i-12)%2==0 else line for i, line in enumerate(latex_str[:-4])] + latex_str[-4:]
latex_str = "\n".join(latex_str)
print(latex_str)


In [ ]:
qmeasures = ["kendalltau"]
# qmeasures = ["pearsonr"]
# qmeasures = ["kendalltau", "spearmanr"]
pivot = data[data["Quality Measure"].isin(qmeasures)].pivot_table(index="Representational Similarity Measure",columns=["Quality Measure", "Setting", "Dataset", "Functional Similarity Measure", "architecture", ],values="corr")
pivot = pivot.apply(abs)
pivot = pivot.sort_values(by="Representational Similarity Measure")
pivot.corr("kendall")
# pivot

In [ ]:
data["Quality Measure"].unique()

In [ ]:
corr_data = data.copy()


## Combined Table (with preliminary values)

In [ ]:
corr_setting = "aug"
corr_qmeasure = "spearmanr"
corr_funcsim = "JSD"
corr_dataset = "mnli"

merge_corr = corr_data.loc[
    (corr_data["Functional Similarity Measure"] == corr_funcsim) &
    (corr_data["Quality Measure"]==corr_qmeasure) &
    (corr_data["Setting"] == corr_setting) &
    (corr_data["Dataset"] == corr_dataset)
]
merge_corr["Score"] = merge_corr.loc[:, "corr"]
merge_corr.loc[:, "Similarity Measure"] = merge_corr.loc[:, "Representational Similarity Measure"]
merge_corr.loc[:, "Setting"] = corr_funcsim+corr_dataset+corr_setting
full_corr_setting = corr_funcsim+corr_dataset+corr_setting
merge_corr.loc[:, "Architecture"] = "BERT-Base"
merge_corr.loc[:, "Quality Metric"] = corr_qmeasure

group_dataset = "mnli"
group_qmeasure = "AUPRC"
merge_groups = groupsep_data.loc[
    (groupsep_data["Dataset"] == group_dataset) &
    (groupsep_data["Quality Metric"] == group_qmeasure)
]

merged = pd.concat([merge_groups, merge_corr], axis=0)
merged


In [ ]:
measure_to_abbrv = {
    "AlignedCosineSimilarity": "AlignCos",
    "CKA": "CKA",
    "ConcentricityDifference": "ConcDiff",
    "DistanceCorrelation": "DistCorr",
    "EigenspaceOverlapScore": "EOS",
    "GeometryScore": "GS",
    "Gulp": "GULP",
    "HardCorrelationMatch": "HardCorr",
    "IMDScore": "IMD",
    "JaccardSimilarity": "Jaccard",
    "LinearRegression": "LinReg",
    "MagnitudeDifference": "MagDiff",
    "OrthogonalAngularShapeMetricCentered": "AngShape",
    "OrthogonalProcrustesCenteredAndNormalized": "OrthProc",
    "PWCCA": "PWCCA",
    "PermutationProcrustes": "PermProc",
    "ProcrustesSizeAndShapeDistance": "ProcDist",
    "RSA": "RSA",
    "RSMNormDifference": "RSMDiff",
    "RankSimilarity": "RankSim",
    "SVCCA": "SVCCA",
    "SecondOrderCosineSimilarity": "2nd-Cos",
    "SoftCorrelationMatch": "SoftCorr",
    "UniformityDifference": "UnifDiff"
}


In [ ]:
fake_vision_data = merged.copy()
fake_vision_data.loc[:, "Architecture"] = "ResNetX"
fake_vision_data.loc[:, "Dataset"] = "ImageNet100"

fake_graph_data = merged.copy()
fake_graph_data.loc[:, "Architecture"] = "GraphSage"
fake_graph_data.loc[:, "Dataset"] = "ogbn-arxiv"

table_data = pd.concat([merged, fake_vision_data, fake_graph_data])
table_data["Similarity Measure"] = table_data["Similarity Measure"].map(measure_to_abbrv)


table_data.loc[table_data.Setting == full_corr_setting, "Score"] = -1 * table_data.loc[table_data.Setting == full_corr_setting, "Score"]

pivot = pd.pivot_table(table_data, index="Similarity Measure",columns=["Setting", "Quality Metric", "Architecture", "Dataset"], values="Score")
pivot = pivot.sort_values(by="Similarity Measure")

styled = pd.io.formats.style.Styler(
    pivot,
    precision=3,
    caption="Results overview for selected datasets and models.",
)

latex_str = styled.to_latex(hrules=True, position="t", label="tab:result_overview", )
latex_str = latex_str.split("\n")
latex_str = [r"\rowcolor{Gray}" + line  if i>=12 and (i-12)%2==0 else line for i, line in enumerate(latex_str[:-4])] + latex_str[-4:]
latex_str = "\n".join(latex_str)
print(latex_str)
